In [1]:
import pandas as pd
from sqlalchemy import create_engine


### Connect to local database

In [6]:
# Update input below

rds_connection_string = "<insert user name>:<insert password>@localhost:5432/customer_db"
engine = create_engine(f'postgresql://{rds_connection_string}')


### Check for tables

In [ ]:
# Load in gun violence / mass shootings tables
mass_shooting = pd.read_excel('Mother Jones - Mass Shootings Database, 1982 - 2019.xlsx')
mass_shooting = mass_shooting[['location', 'fatalities', 'injured', 'date', 'case', 'age_of_shooter']].copy()

mass_shooting.head()

In [ ]:
mass_shooting['city'] = mass_shooting['location'].str.split(', ').str[0]
mass_shooting['state'] = mass_shooting['location'].str.split(', ').str[1]
mass_shooting= mass_shooting.rename(columns={"city": "City", "state" : "State"})

mass_shooting.head()

In [ ]:
gun_violence = "gun_violence.csv"
gun_violence_df = pd.read_csv(gun_violence)
gun_violence_df.rename(columns={'state':'State'}, inplace=True)
gun_violence_df = gun_violence_df[['State', 'city_or_county', 'n_killed', 'n_injured', 'date', 'participant_name', 'participant_age']].copy()
gun_violence_df= gun_violence_df.rename(columns={"city_or_county": "City", "n_killed" : "fatalities", "n_injured": :"injured", "participant_age": "age_of_shooter"})
gun_violence_df.head()

In [6]:
# load in cities table
cities = pd.read_excel('Resources/US City Populations.xlsx')
#gun_violence = pd.read_csv('Resources/gun-violence-data_01-2013_03-2018.csv')
cities.head()

,State,City,Population
0,Alabama,Abernant,7599
1,Alabama,Alabaster,31545
2,Alabama,Albertville,21458
3,Alabama,Alexander City,14849
4,Alabama,Andalusia,9081


In [7]:
# Process cities table to generate per capita information

gun_v = gun_violence[['State', 'City']]


gun_v['count'] = gun_v.groupby(['State', 'City'])['State'].transform('size')

#gun_v.rename(columns={'city_or_county':'City'}, inplace=True) 
#gun_v.rename(columns={'state':'State'}, inplace=True)

gun_v2 = pd.merge(gun_v, cities, on=['State', 'City'], how='left')

gun_v2 = gun_v2.drop_duplicates()

gun_v2['shootings_per_cap'] = gun_v2['count']/gun_v2['Population']

us_state_abbrev = {
   'Alabama': 'AL',
   'Alaska': 'AK',
   'Arizona': 'AZ',
   'Arkansas': 'AR',
   'California': 'CA',
   'Colorado': 'CO',
   'Connecticut': 'CT',
   'Delaware': 'DE',
   'District of Columbia': 'DC',
   'Florida': 'FL',
   'Georgia': 'GA',
   'Hawaii': 'HI',
   'Idaho': 'ID',
   'Illinois': 'IL',
   'Indiana': 'IN',
   'Iowa': 'IA',
   'Kansas': 'KS',
   'Kentucky': 'KY',
   'Louisiana': 'LA',
   'Maine': 'ME',
   'Maryland': 'MD',
   'Massachusetts': 'MA',
   'Michigan': 'MI',
   'Minnesota': 'MN',
   'Mississippi': 'MS',
   'Missouri': 'MO',
   'Montana': 'MT',
   'Nebraska': 'NE',
   'Nevada': 'NV',
   'New Hampshire': 'NH',
   'New Jersey': 'NJ',
   'New Mexico': 'NM',
   'New York': 'NY',
   'North Carolina': 'NC',
   'North Dakota': 'ND',
   'Northern Mariana Islands':'MP',
   'Ohio': 'OH',
   'Oklahoma': 'OK',
   'Oregon': 'OR',
   'Palau': 'PW',
   'Pennsylvania': 'PA',
   'Puerto Rico': 'PR',
   'Rhode Island': 'RI',
   'South Carolina': 'SC',
   'South Dakota': 'SD',
   'Tennessee': 'TN',
   'Texas': 'TX',
   'Utah': 'UT',
   'Vermont': 'VT',
   'Virgin Islands': 'VI',
   'Virginia': 'VA',
   'Washington': 'WA',
   'West Virginia': 'WV',
   'Wisconsin': 'WI',
   'Wyoming': 'WY',
}
states= pd.DataFrame([us_state_abbrev]).transpose().reset_index()
#states.head()
state_ab= states.rename(columns={"index": "State", 0 : "abbrev"})

gun_v3 = pd.merge(gun_v2, state_ab, on='State', how='left')

C:\Users\Alex\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [8]:
gun_v3.head()

,State,City,count,Population,shootings_per_cap,abbrev
0,Pennsylvania,Mckeesport,84,NaN,NaN,PA
1,California,Hawthorne,21,87583.0,0.000240,CA
2,Ohio,Lorain,113,63776.0,0.001772,OH
3,Colorado,Aurora,166,353108.0,0.000470,CO
4,North Carolina,Greensboro,522,282586.0,0.001847,NC


In [7]:
engine.table_names()


['customer_location', 'customer_name']

### Use pandas to load csv converted Gun Violence DataFrame into database

In [8]:
gun_violence_df.to_sql(name='FROM ABOVE', con=engine, if_exists='append', index=False)


### Use pandas to load csv converted Mass Shooting DataFrame into database

In [1]:
mass_shooting_df.to_sql(name='ALSO FROM ABOVE', con=engine, if_exists='append', index=False)


NameError: name 'mass_shooting_df' is not defined

### Use pandas to load csv converted City Population DataFrame into database

In [ ]:
city_population_df.to_sql(name='ALSO FROM ABOVE', con=engine, if_exists='append', index=False)


### Confirm data has been added by querying the FROM ABOVE table
* NOTE: can also check using pgAdmin

In [10]:
pd.read_sql_query('select * from FROM ABOVE', con=engine).head()


,id,first_name,last_name
0,1,Benetta,Cancott
1,2,Lilyan,Cherry
2,3,Ezekiel,Benasik
3,4,Kennedy,Atlay
4,5,Sanford,Salmen


### Confirm data has been added by querying the ALSO FROM ABOVE table

In [11]:
pd.read_sql_query('select * from ALSO FROM ABOVE', con=engine).head()


,id,address,us_state
0,1,043 Mockingbird Place,Indiana
1,2,4 Prentice Point,Indiana
2,3,46 Derek Junction,Texas
3,4,11966 Old Shore Place,Missouri
4,5,5 Evergreen Circle,New York


### Confirm data has been added by querying the ALSO FROM ABOVE 2 table

In [ ]:
pd.read_sql_query('select * from ALSO FROM ABOVE2', con=engine).head()
